In [1]:
import os
os.chdir("C:\\Users\\Shrivatav\\Desktop\\Deep Learning A-Z\\Deep_Learning_A_Z\\Volume 2 - Unsupervised Deep Learning\\Part 5 - Boltzmann Machines (BM)\\Section 19 - Building a Boltzmann Machine\\Boltzmann_Machines")

In [2]:
os.getcwd()

'C:\\Users\\Shrivatav\\Desktop\\Deep Learning A-Z\\Deep_Learning_A_Z\\Volume 2 - Unsupervised Deep Learning\\Part 5 - Boltzmann Machines (BM)\\Section 19 - Building a Boltzmann Machine\\Boltzmann_Machines'

In [3]:
import numpy as np
import pandas as pd
import torch

In [4]:
# Import elements from pytorch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [5]:
# Import dataset
movies = pd.read_csv("ml-1m/movies.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")

In [6]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies.shape

(3883, 3)

In [8]:
# Import users data
users = pd.read_csv("ml-1m/users.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")

In [9]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [10]:
# Import ratings
ratings = movies = pd.read_csv("ml-1m/ratings.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
ratings.shape

(1000209, 4)

For ratings data:  
Col 0 - User ID  
Col 1 - Movie ID  
Col 2 - Rating  
Col 3 - Timestamp

In [11]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [12]:
# Import training and test sets
training_set1 = pd.read_csv("ml-100k/u1.base", delimiter = "\t", header = None)
training_set1.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [13]:
training_set1.shape

(80000, 4)

In [14]:
# Convert to array
training_set = np.array(training_set1, dtype = 'int')

In [15]:
training_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [16]:
# Same for test data
test_set = pd.read_csv("ml-100k/u1.test", delimiter = "\t", header = None)
test_set = np.array(test_set, dtype = 'int')
test_set

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [17]:
# Calculate total number of users and total number of movies
#nb_users = max(max(training_set[:,0]), max(test_set[:,0])) # Col 0 is user id

In [18]:
#nb_users

In [19]:
#nb_movies = max(max(training_set[:,1]), max(test_set[:,1])) # Col 1 is movie id
#nb_movies

In [20]:
# Find the total number of users and movies in the full dataset (training + test)
combined = np.append(training_set, test_set, axis=0)
nb_users = len(np.unique(combined[:,0]))
nb_movies = len(np.unique(combined[:,1]))
print(nb_users)
print(nb_movies)

943
1682


In [21]:
# Convert data into array such that users are rows and movies are columns
# Torch takes input as list of lists
def convert(data):
    # Create list of lists
    # Each element is a list of all ratings
    # where rating at ith index belongs to ith movie
    new_data = []
    
    for user_id in range(1, nb_users + 1):
        movie_id = data[:,1][data[:,0] == user_id] # all movies for this user
        rating = data[:,2][data[:,0] == user_id] # all ratings for this user
        ratings = np.zeros(nb_movies)
        ratings[movie_id - 1] = rating # Indexing starts at zero but first movie id is 1
        new_data.append(ratings)
    
    return new_data

In [22]:
training_set = convert(training_set)
test_set = convert(test_set)

In [23]:
training_set

[array([5., 3., 4., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([3., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 2., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([3., 0., 0., ..., 

In [24]:
test_set

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 3., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 3., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([3., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 

In [25]:
# Convert to tensors for torch inputs
training_set_tensor = torch.FloatTensor(training_set)
test_set_tensor = torch.FloatTensor(test_set)

In [26]:
training_set_tensor

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [27]:
# Convert ratings to binary
# 1 = Liked the movies
# 0 - Didn't like the movie

# Replace 0s to -1s as they are ratings for movies that were not rated
training_set_tensor[training_set_tensor == 0] = -1

# Encode ratings of 1 or 2 as not liked (0)
training_set_tensor[training_set_tensor == 1] = 0
training_set_tensor[training_set_tensor == 2] = 0

# Encode ratings of 3,4,5 as liked (1)
training_set_tensor[training_set_tensor >= 3] = 1

In [28]:
# Convert ratings to binary
# 1 = Liked the movies
# 0 - Didn't like the movie

# Replace 0s to -1s as they are ratings for movies that were not rated
test_set_tensor[test_set_tensor == 0] = -1

# Encode ratings of 1 or 2 as not liked (0)
test_set_tensor[test_set_tensor == 1] = 0
test_set_tensor[test_set_tensor == 2] = 0

# Encode ratings of 3,4,5 as liked (1)
test_set_tensor[test_set_tensor >= 3] = 1

In [29]:
test_set_tensor

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

In [30]:
training_set_tensor

tensor([[ 1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

In [64]:
# Build a class to define architecture of RBM

class RBM():
    def __init__(self, n_visible, n_hidden):
        # Initialise weights
        # Create a tensor of normally distributed (mean = 0, variance = 1)
        # random weights of size n_hidden * n_visible
        self.Weights = torch.randn(n_hidden, n_visible)
        # Bias for probabilities of hidden nodes given visible nodes
        self.a = torch.randn(1, n_hidden)
        # Bias for probabilities of visible nodes given given nodes
        self.b = torch.randn(1, n_visible)
    
    # Function to give probabilities of hidden nodes given visible nodes
    def sample_h(self, x):
        # Product of x visbile nodes and tensor of weights
        wx = torch.mm(x, self.Weights.t())
        # Add bias
        activation = wx + self.a.expand_as(wx)
        # Vector of probabilities such that ith element of p_h_given_v is 
        # the probability that ith hidden node is activated
        p_h_given_v = torch.sigmoid(activation)
        
        # Return bernoulli sampled numbers 0 or 1 depending upon whether the
        # hidden neuron got activated or not
        
        # Bernoulli sampling generates a random number for a probability (say 0.7)
        # and returns 1 if that random number is greater than that probability
        # and zero otherwise
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    # Function to give probabilities of visible nodes given hidden nodes
    def sample_v(self, y):
        # Product of y visbile nodes and tensor of weights
        wy = torch.mm(y, self.Weights)
        # Add bias
        activation = wy + self.b.expand_as(wy)
        # Vector of probabilities such that ith element of p_v_given_h is 
        # the probability that ith hidden node is activated
        p_v_given_h = torch.sigmoid(activation)
        
        # Return bernoulli sampled numbers 0 or 1 depending upon whether the
        # hidden neuron got activated or not
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    # Update tensor of weights and biases
    # using contrastive divergence algortithm
    def train(self, v0, vk, ph0, phk):
        '''
        v0: input vector with all ratings given by one user
        vk: visible nodes after k samplings (k round trips from visibile to hidden and then hidden to visible)
        ph0: initial vector of probabilities that hidden nodes = 1 given v0
        phk: vector of probabilities after k samplings that hidden nodes = 1 given vk
        '''
        
        # Update weights:
        #self.Weights += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.Weights += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [65]:
# Number of visible nodes = number of ratings
nv = len(training_set[0])
# Number of hidden nodes (change and tune later)
nh = 100
# Batch size - update weights after number of observations equal to batch size
batch_size = 100

In [66]:
# Create RBM object
rbm = RBM(n_visible = nv, n_hidden = nh)

In [67]:
# Train the RBM
n_epoch = 10

for epoch in range(1, n_epoch + 1):
    train_loss = 0
    counter = 0.0 # used for normalising
    
    # Loop over each user in steps of batch_size
    for user in range(0, nb_users - batch_size, batch_size):
        vk = training_set_tensor[user:user+batch_size] # batch of batch_size from current user
        v0 = training_set_tensor[user:user+batch_size]
        ph0, _ = rbm.sample_h(v0)
        
        # Random walk for contrastive divergence
        for k in range(n_epoch):
            # hk = hidden nodes at second step of contrastive divergence
            _, hk = rbm.sample_h(vk)
            # sampled visible nodes after first step of Gibbs sampling
            _, vk = rbm.sample_v(hk)
            # Reset -1 values in original vector of visible nodes
            vk[v0 < 0] = v0[v0 < 0]
        
        phk, _ = rbm.sample_h(vk)
        
        # Train
        rbm.train(v0, vk, ph0, phk)
        # Update train_loss
        # Use distance to measure error
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        counter += 1.0
    print('Epoch: ' + str(epoch) + ' loss: ' + str(train_loss/counter))

Epoch: 1 loss: tensor(0.3473)
Epoch: 2 loss: tensor(0.2482)
Epoch: 3 loss: tensor(0.2435)
Epoch: 4 loss: tensor(0.2512)
Epoch: 5 loss: tensor(0.2463)
Epoch: 6 loss: tensor(0.2504)
Epoch: 7 loss: tensor(0.2534)
Epoch: 8 loss: tensor(0.2481)
Epoch: 9 loss: tensor(0.2488)
Epoch: 10 loss: tensor(0.2478)


In [68]:
# Testing the RBM
test_loss = 0
counter = 0.0 # used for normalising
    
# Loop over each user
for user in range(nb_users):
    v = training_set_tensor[user:user+1] # use ratings from train to activate hidden neurons of RBM
    vt = test_set_tensor[user:user+1]
    
    if len(vt[vt >= 0]) > 0:
        # h = hidden nodes at second step of contrastive divergence
        _, h = rbm.sample_h(v)
        # sampled visible nodes after first step of Gibbs sampling
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        counter += 1.0
    
print(' loss: ' + str(test_loss/counter))

 loss: tensor(0.2515)


In [ ]:
# Use RMSE to evaluate the model
'''
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2)) # RMSE here
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
Test phase:

test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2)) # RMSE here
        s += 1.
print('test loss: '+str(test_loss/s))
'''